In [ ]:
!pip install numpy
!pip install pandas 
!pip install matplotlib

In [9]:
import os
import pandas as pd
import numpy as np

# =============================================================================
# BLOCO PRINCIPAL DE CARREGAMENTO DE DADOS
# =============================================================================

if __name__ == "__main__":
    
    # --- 1. CONFIGURAÇÃO ---
    
    # INSTRUÇÃO: Altere esta linha para o caminho da sua pasta principal "DATASET".
    # Usamos r'' para criar uma "raw string", que trata as barras invertidas do Windows corretamente.
    caminho_raiz = r'C:\Users\vinic\OneDrive\Documentos\Graduação\TG\Dataset'

    # Dicionários para mapear informações dos nomes dos arquivos
    mapa_tipo_falha = {'IR': 'Pista Interna', 'B': 'Esfera', 'OR': 'Pista Externa', 'Normal': 'Normal'}
    mapa_diametro_falha = {'7': '0.007"', '14': '0.014"', '21': '0.021"'}
    mapa_rpm_para_carga = {'1797': 0, '1772': 1, '1750': 2, '1730': 3}

    # --- 2. PROCESSAMENTO E CARREGAMENTO ---

    # Dicionário para guardar cada DataFrame individualmente
    dicionario_de_dataframes = {}
    print(f"Iniciando a leitura dos arquivos em '{caminho_raiz}'...")

    # Navega por todas as pastas e arquivos a partir do caminho raiz
    for pasta_atual, _, arquivos in os.walk(caminho_raiz):
        for nome_arquivo in arquivos:
            if nome_arquivo.endswith('.npz'):
                caminho_completo = os.path.join(pasta_atual, nome_arquivo)
                
                # Decodifica o nome do arquivo para extrair metadados
                nome_sem_ext = nome_arquivo.replace('.npz', '')
                partes = nome_sem_ext.split('_')
                
                rpm = partes[0]
                carga_hp = mapa_rpm_para_carga.get(rpm, -1)

                if 'Normal' in nome_arquivo:
                    tipo_falha_cod, diametro_falha = 'Normal', 'N/A'
                    sensores_para_processar = ['DE', 'FE'] 
                else:
                    tipo_falha_cod = partes[1].split('@')[0]
                    diametro_falha = mapa_diametro_falha.get(partes[2], 'Desconhecido')
                    sensores_para_processar = ['DE'] if 'DE' in partes[-1] else ['FE']

                tipo_falha = mapa_tipo_falha.get(tipo_falha_cod, 'Desconhecido')

                try:
                    dados_npz = np.load(caminho_completo)
                    for sensor_cod in sensores_para_processar:
                        if sensor_cod in dados_npz.files:
                            # Define uma chave única para o dicionário, removendo a taxa de amostragem do nome.
                            if 'Normal' in nome_arquivo:
                                # Para arquivos normais, a chave não tem taxa, apenas adicionamos o nome do sensor.
                                chave_df = f"{nome_sem_ext}_{sensor_cod}"
                            else:
                                # Para arquivos de falha, removemos a taxa do nome original para criar a chave.
                                # Ex: '1730_B_7_DE12' se torna '1730_B_7_DE'
                                partes_chave = nome_sem_ext.split('_')
                                partes_chave[-1] = partes_chave[-1].rstrip('0123456789')
                                chave_df = "_".join(partes_chave)

                            # Cria o DataFrame com os dados de amplitude
                            df_temp = pd.DataFrame({'amplitude': dados_npz[sensor_cod].ravel()})
                            
                            # Adiciona as colunas com os metadados
                            df_temp['arquivo_origem'] = nome_arquivo
                            df_temp['rotacao_rpm'] = int(rpm)
                            df_temp['carga_motor_hp'] = carga_hp
                            df_temp['tipo_falha'] = tipo_falha
                            df_temp['diametro_falha'] = diametro_falha
                            df_temp['local_sensor'] = 'Drive End' if sensor_cod == 'DE' else 'Fan End'
                            
                            # Adiciona o DataFrame ao dicionário usando a chave única
                            dicionario_de_dataframes[chave_df] = df_temp
                except Exception as e:
                    print(f"Erro ao processar o arquivo {nome_arquivo}: {e}")

    # --- 3. VERIFICAÇÃO FINAL ---
    if not dicionario_de_dataframes:
        print("\nERRO: Nenhum arquivo de dados foi carregado.")
        print(f"Por favor, verifique se a variável 'caminho_raiz' ('{caminho_raiz}') está correta.")
    else:
        print("\nProcesso de carregamento concluído!")
        print(f"Total de {len(dicionario_de_dataframes)} conjuntos de dados carregados no dicionário.")
        
        # Exibe informações para verificação
        print("\n--- Chaves de Exemplo no Dicionário ---")
        # Pega as 5 primeiras chaves para mostrar como exemplo
        chaves_exemplo = list(dicionario_de_dataframes.keys())[:5]
        for chave in chaves_exemplo:
            print(f"- '{chave}'")

        print("\n--- Exemplo de Acesso a um DataFrame ---")
        if chaves_exemplo:
            chave_para_mostrar = chaves_exemplo[0]
            print(f"Acessando o DataFrame com a chave: '{chave_para_mostrar}'")
            print(dicionario_de_dataframes[chave_para_mostrar].head())



Iniciando a leitura dos arquivos em 'C:\Users\vinic\OneDrive\Documentos\Graduação\TG\Dataset'...

Processo de carregamento concluído!
Total de 75 conjuntos de dados carregados no dicionário.

--- Chaves de Exemplo no Dicionário ---
- '1730_B_14_DE'
- '1730_B_14_FE'
- '1730_B_21_DE'
- '1730_B_21_FE'
- '1730_B_7_DE'

--- Exemplo de Acesso a um DataFrame ---
Acessando o DataFrame com a chave: '1730_B_14_DE'
   amplitude      arquivo_origem  rotacao_rpm  carga_motor_hp tipo_falha  \
0   0.105420  1730_B_14_DE12.npz         1730               3     Esfera   
1  -0.107370  1730_B_14_DE12.npz         1730               3     Esfera   
2  -0.163410  1730_B_14_DE12.npz         1730               3     Esfera   
3   0.118903  1730_B_14_DE12.npz         1730               3     Esfera   
4   0.184039  1730_B_14_DE12.npz         1730               3     Esfera   

  diametro_falha local_sensor  
0         0.014"    Drive End  
1         0.014"    Drive End  
2         0.014"    Drive End  
3      

In [53]:
# =============================================================================
# BLOCO DE VERIFICAÇÃO PARA DADOS DO DRIVE END (DE)
# =============================================================================

print("\n\n--- INICIANDO VERIFICAÇÃO DE INTEGRIDADE PARA O DRIVE END ---")

# Passo 1: Obter todas as chaves de falha do Drive End que foram CARREGADAS
# Filtramos para pegar apenas chaves que contêm 'DE' e não são 'Normal'
chaves_de_carregadas = [
    chave for chave, df in dicionario_de_dataframes.items()
    if 'DE' in chave and df['tipo_falha'].iloc[0] != 'Normal'
]

# Passo 2: Obter todas as chaves de falha do Drive End que foram PROCESSADAS com sucesso
# A coluna 'chave' no df_resultados_de contém essa informação
if not df_resultados_de.empty:
    chaves_de_processadas = df_resultados_de['chave'].unique().tolist()
else:
    chaves_de_processadas = []

# Passo 3: Comparar as duas listas para encontrar as chaves que faltam
# Usar 'sets' é a maneira mais eficiente de encontrar a diferença entre duas listas
set_carregadas = set(chaves_de_carregadas)
set_processadas = set(chaves_de_processadas)

chaves_faltando = sorted(list(set_carregadas - set_processadas))

# --- Relatório Final ---
if not chaves_faltando:
    print(f"VERIFICAÇÃO CONCLUÍDA: Sucesso! Todas as {len(chaves_de_carregadas)} chaves de falha do Drive End carregadas foram processadas.")
else:
    print(f"ATENÇÃO: {len(chaves_faltando)} de {len(chaves_de_carregadas)} chaves do Drive End foram carregadas, mas NÃO aparecem nos resultados finais.")
    print("Isso pode ocorrer porque o pico de falha não foi encontrado (verifique a tolerância) ou por outra razão.")
    print("\nLista de chaves faltando:")
    for chave in chaves_faltando:
        print(f"- {chave}")



--- INICIANDO VERIFICAÇÃO DE INTEGRIDADE PARA O DRIVE END ---
VERIFICAÇÃO CONCLUÍDA: Sucesso! Todas as 37 chaves de falha do Drive End carregadas foram processadas.


# Frequências Características de falha
**Frequência devido à falha local na pista externa ($f_e$):**
$$
f_e = \frac{n f_r}{2} \left( 1 - \frac{d}{D} \cos \phi \right)
$$

**Frequência devido à falha local na pista interna ($f_i$):**
$$
f_i = \frac{n f_r}{2} \left( 1 + \frac{d}{D} \cos \phi \right)
$$

**Frequência fundamental da carcaça ($f_C$):**
$$
f_C = \frac{f_r}{2} \left( 1 - \frac{d}{D} \cos \phi \right)
$$

**Frequência devido à falha na esfera ($f_B$):**
$$
f_B = \frac{D f_r}{2d} \left[ 1 - \left( \frac{d}{D} \cos \phi \right)^2 \right]
$$

In [ ]:
# Importando bibliotecas necessárias
import numpy as np
import pprint # Usaremos para imprimir os dicionários de forma mais legível


# Passo 2: Definir os parâmetros e a lista de rotações
# Fonte: https://engineering.case.edu/bearingdatacenter/bearing-information

# Lista de rotações dos experimentos
lista_rpm = [1797, 1772, 1750, 1730]

# Parâmetros do rolamento do LADO DO ACIONAMENTO (Drive End)
params_drive_end = {
    'n': 9, 'd': 0.3126, 'D': 1.537, 'phi_graus': 0.0
}

# Parâmetros do rolamento do LADO DO VENTILADOR (Fan End)
params_fan_end = {
    'n': 8, 'd': 0.2500, 'D': 1.122, 'phi_graus': 0.0
}

TAXA_AMOSTRAL = 12000


In [64]:
# Este script assume que as seguintes variáveis já existem no seu ambiente:
# - dicionario_de_dataframes: Dicionário com os dados carregados.
# - params_drive_end, params_fan_end: Dicionários com parâmetros dos rolamentos.
# - TAXA_AMOSTRAL: A taxa de amostragem.

import pandas as pd
import numpy as np

# =============================================================================
# FUNÇÕES DE ANÁLISE (mesmas funções de antes)
# =============================================================================

def calcular_frequencias_rolamento(n, fr, d, D, phi_graus=0.0):
    phi_rad = np.deg2rad(phi_graus)
    termo_comum = (d / D) * np.cos(phi_rad)
    resultados = {
        'Pista Externa': (n * fr / 2) * (1 - termo_comum),
        'Pista Interna': (n * fr / 2) * (1 + termo_comum),
        'Carcaça': (fr / 2) * (1 - termo_comum),
        'Esfera': (D * fr / (2 * d)) * (1 - termo_comum**2)
    }
    return resultados

def encontrar_amplitude_pico(sinal_temporal, taxa_amostral, freq_alvo, tolerancia_hz=3.0):
    N = len(sinal_temporal)
    if N == 0: return None, None
    yf = np.fft.fft(sinal_temporal)
    amplitude_espectral = 2.0/N * np.abs(yf[0:N//2])
    xf = np.fft.fftfreq(N, 1 / taxa_amostral)[:N//2]
    indices_janela = np.where((xf >= freq_alvo - tolerancia_hz) & (xf <= freq_alvo + tolerancia_hz))
    if len(indices_janela[0]) == 0: return None, None
    amplitude_pico = np.max(amplitude_espectral[indices_janela])
    indice_pico = np.argmax(amplitude_espectral[indices_janela])
    frequencia_pico = xf[indices_janela][indice_pico]
    return frequencia_pico, amplitude_pico

# =============================================================================
# BLOCO DE ANÁLISE (MODIFICADO)
# =============================================================================

print("--- INICIANDO ANÁLISE DE FALHAS (TODOS OS DIÂMETROS) ---")
resultados_analise = []

# Loop principal para analisar cada arquivo no dicionário
for chave, df in dicionario_de_dataframes.items():
    # Pula os casos normais, analisa apenas os casos de falha
    if df['tipo_falha'].iloc[0] != 'Normal':
        
        # Extrai metadados do DataFrame
        rpm = df['rotacao_rpm'].iloc[0]
        fr_hz = rpm / 60
        tipo_falha = df['tipo_falha'].iloc[0].strip()
        diametro_falha = df['diametro_falha'].iloc[0]
        local_sensor = df['local_sensor'].iloc[0]
        sinal = df['amplitude'].values
        
        # Seleciona parâmetros do rolamento correto
        params_rolamento = params_drive_end if local_sensor == 'Drive End' else params_fan_end
        
        # Calcula frequências teóricas
        freqs_teoricas = calcular_frequencias_rolamento(fr=fr_hz, **params_rolamento)
        freq_alvo = freqs_teoricas.get(tipo_falha)
        
        if not freq_alvo: continue

        # Encontra o pico de amplitude no sinal
        freq_pico, amp_pico = encontrar_amplitude_pico(sinal, TAXA_AMOSTRAL, freq_alvo)
        
        # Se um pico for encontrado, armazena os resultados
        if freq_pico is not None:
            resultados_analise.append({
                'chave': chave,
                'tipo_falha': tipo_falha,
                'diametro_falha': diametro_falha,
                'rpm': rpm,
                'local_sensor': local_sensor,
                'freq_teorica_hz': freq_alvo,
                'freq_encontrada_hz': freq_pico,
                'amplitude_falha': amp_pico
            })

# --- GERAÇÃO DOS DATAFRAMES FINAIS ---

# Cria o DataFrame geral com todos os resultados
df_resultados_geral = pd.DataFrame(resultados_analise)

# Separa os resultados por local do sensor (DE e FE)
df_resultados_de = df_resultados_geral[df_resultados_geral['local_sensor'] == 'Drive End'].copy()
df_resultados_fe = df_resultados_geral[df_resultados_geral['local_sensor'] == 'Fan End'].copy()

#print("\n--- Análise Concluída. Resumo dos Resultados ---")
#print("\n--- Resultados do Drive End (DE) ---")
#print(df_resultados_de.head())

#print("\n--- Resultados do Fan End (FE) ---")
#print(df_resultados_fe.head())


# =============================================================================
# BLOCO DE COMPARAÇÃO (EXEMPLO)
# =============================================================================

print("\n\n--- COMPARAÇÃO DE AMPLITUDE DE FALHA ---")
print("Falha na Pista Interna a 1730 RPM para diâmetros diferentes (Sensor Fan End)")

# Filtra o DataFrame do Fan End para a condição específica
comparacao_exemplo = df_resultados_fe.query("rpm == 1730 and tipo_falha == 'Pista Interna'")

# Seleciona e renomeia colunas para uma visualização clara
comparacao_exemplo = comparacao_exemplo[['diametro_falha', 'amplitude_falha', 'freq_teorica_hz', 'freq_encontrada_hz']]

# Ordena os resultados pelo diâmetro da falha
comparacao_exemplo = comparacao_exemplo.sort_values(by='diametro_falha').reset_index(drop=True)

print(comparacao_exemplo)


print("\n\n--- COMPARAÇÃO DE RPM ---")
print("Falha na Pista Interna para diâmetro de falha de 0.014 (Sensor Fan End)")

# Filtra o DataFrame do Fan End para a condição específica
comparacao_exemplo = df_resultados_fe.query("diametro_falha == '0.014\"' and tipo_falha == 'Pista Interna'")

# Seleciona e renomeia colunas para uma visualização clara
comparacao_exemplo = comparacao_exemplo[['rpm', 'amplitude_falha', 'freq_teorica_hz', 'freq_encontrada_hz']]

# Ordena os resultados pelo diâmetro da falha
comparacao_exemplo = comparacao_exemplo.sort_values(by='rpm').reset_index(drop=True)

print(comparacao_exemplo)


--- INICIANDO ANÁLISE DE FALHAS (TODOS OS DIÂMETROS) ---


--- COMPARAÇÃO DE AMPLITUDE DE FALHA ---
Falha na Pista Interna a 1730 RPM para diâmetros diferentes (Sensor Fan End)
  diametro_falha  amplitude_falha  freq_teorica_hz  freq_encontrada_hz
0         0.007"         0.009931       141.031491          142.378739
1         0.014"         0.009651       141.031491          142.512875
2         0.021"         0.009933       141.031491          142.809983


--- COMPARAÇÃO DE RPM ---
Falha na Pista Interna para diâmetro de falha de 0.014 (Sensor Fan End)
    rpm  amplitude_falha  freq_teorica_hz  freq_encontrada_hz
0  1730         0.009651       141.031491          142.512875
1  1750         0.009640       142.661913          144.374583
2  1772         0.008299       144.455377          145.956770
3  1797         0.009396       146.493405          147.637844


# Amplitudes de Falha em função do diâmetro da falha

#### Falha na Esfera

1730 RPM Esfera (Sensor Drive End)

| diametro_falha | amplitude_falha | freq_encontrada_hz |
| :------------- | :-------------- | :----------------- |
| 0.007"         | 0.000076        | 68.906512          |
| 0.014"         | 0.000052        | 67.007271          |
| 0.021"         | 0.000061        | 67.203773          |


Diâmetro de falha de 0.014" (Sensor Drive End)

| rpm | amplitude_falha | freq_encontrada_hz |
| :--- | :--- | :--- |
| 1730 | 0.000052 | 67.007271 |
| 1750 | 0.000048 | 66.988548 |
| 1772 | 0.000038 | 70.937316 |
| 1797 | 0.000027 | 71.894030 |


#### Falha na Pista Interna

Falha na Pista Interna a 1730 RPM para diâmetros diferentes (Sensor Drive End)

| diametro_falha | amplitude_falha | freq_encontrada_hz |
| :--- | :--- | :--- |
| 0.007" | 0.005900 | 154.933817 |
| 0.014" | 0.008288 | 155.495846 |
| 0.021" | 0.011044 | 155.421302 |


Diâmetro de falha de 0.014" (Sensor Drive End)

| rpm | amplitude_falha | freq_encontrada_hz |
| :--- | :--- | :--- |
| 1730 | 0.008288 | 155.495846 |
| 1750 | 0.009055 | 157.674442 |
| 1772 | 0.009426 | 159.644141 |
| 1797 | 0.007026 | 161.712325 |

#### Pista Externa

Falha na Pista Externa a 1730 RPM para diâmetros diferentes (Sensor Drive End)

| diametro_falha | amplitude_falha | freq_encontrada_hz |
| :--- | :--- | :--- |
| 0.007" | 0.002440 | 103.384977 |
| 0.014" | 0.000360 | 103.286308 |
| 0.021" | 0.002292 | 102.991204 |


Falha na Pista Externa para diâmetro de falha de 0.014" (Sensor Drive End)

| rpm | amplitude_falha | freq_encontrada_hz |
| :--- | :--- | :--- |
| 1730 | 0.000360 | 103.286308 |
| 1750 | 0.000369 | 104.886496 |
| 1772 | 0.000420 | 106.307723 |
| 1797 | 0.000364 | 107.644075 |


In [59]:
import numpy as np
import pandas as pd

# =============================================================================
# PARÂMETROS DE GERAÇÃO (COM A ALTERAÇÃO)
# =============================================================================

# Amplitudes de referência específicas para cada rolamento (DE e FE)
# ESTRUTURA ANINHADA: Local do Sensor -> Tipo da Falha -> Amplitude
# ATENÇÃO: Os valores para 'Fan End' são exemplos, ajuste conforme sua análise.
amplitudes_referencia = {
    'Drive End': {
        'Esfera': 0.000052,
        'Pista Interna': 0.0083,
        'Pista Externa': 0.00036
    },
    'Fan End': {
        'Esfera': 0.0029,        # Exemplo de valor diferente para FE
        'Pista Interna': 0.0096,   # Exemplo de valor diferente para FE
        'Pista Externa': 0.0031    # Exemplo de valor diferente para FE
    }
}

# Os outros parâmetros permanecem os mesmos
multiplicadores = [0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 50.0]
# TAXA_AMOSTRAL = 12000 # Deve estar definida

# =============================================================================
# PASSO 1: IDENTIFICAR OS SINAIS NORMAIS DE BASE
# =============================================================================

dicionario_sinais_normais = {
    chave: df for chave, df in dicionario_de_dataframes.items()
    if df['tipo_falha'].iloc[0] == 'Normal'
}
print(f"Encontrados {len(dicionario_sinais_normais)} sinais de operação normal para usar como base.")

# =============================================================================
# PASSO 2 E 3: GERAR FALHAS E SOMAR AOS SINAIS NORMAIS
# =============================================================================

lista_dados_finais = []

for chave_normal, df_normal in dicionario_sinais_normais.items():
    sinal_normal_base = df_normal['amplitude'].values
    N_PONTOS = len(sinal_normal_base)
    rpm_atual = df_normal['rotacao_rpm'].iloc[0]
    local_sensor = df_normal['local_sensor'].iloc[0]
    arquivo_origem = df_normal['arquivo_origem'].iloc[0]
    
    print(f"\nProcessando base: {arquivo_origem} ({N_PONTOS} pontos, {rpm_atual} RPM, Sensor {local_sensor})")
    
    df_resultados_fonte = df_resultados_de if local_sensor == 'Drive End' else df_resultados_fe
    
    duracao_real_s = N_PONTOS / TAXA_AMOSTRAL
    t = np.linspace(0.0, duracao_real_s, N_PONTOS, endpoint=False)
    
    for tipo_falha in ['Pista Externa', 'Pista Interna', 'Esfera']:
        try:
            freq_teorica = df_resultados_fonte.query(
                f"rpm == {rpm_atual} and tipo_falha == '{tipo_falha}'"
            )['freq_teorica_hz'].iloc[0]
        except IndexError:
            print(f"  - Aviso: Frequência não encontrada para {tipo_falha} a {rpm_atual} RPM. Pulando.")
            continue
            
        # --- ALTERAÇÃO PRINCIPAL AQUI ---
        # Acessa o dicionário aninhado usando o local do sensor e o tipo da falha
        amp_ref = amplitudes_referencia[local_sensor][tipo_falha]
        
        for mult in multiplicadores:
            amplitude_final = amp_ref * mult
            sinal_falha_sintetico = amplitude_final * np.sin(2 * np.pi * freq_teorica * t)
            sinal_final_combinado = sinal_normal_base + sinal_falha_sintetico
            
            lista_dados_finais.append({
                'sinal_final': sinal_final_combinado,
                'tipo_falha_adicionada': tipo_falha,
                'rpm': rpm_atual,
                'multiplicador_amplitude': mult,
                'arquivo_normal_base': arquivo_origem,
                'local_sensor': local_sensor
            })

# =============================================================================
# PASSO 4: MONTAR OS DATAFRAMES FINAIS
# =============================================================================

df_final_geral = pd.DataFrame(lista_dados_finais)
df_final_de = df_final_geral[df_final_geral['local_sensor'] == 'Drive End'].reset_index(drop=True)
df_final_fe = df_final_geral[df_final_geral['local_sensor'] == 'Fan End'].reset_index(drop=True)

print("\n\n--- Geração e Combinação Concluídas! ---")
print(f"Total de {len(df_final_de)} sinais com falha gerados para o Drive End.")
print(f"Total de {len(df_final_fe)} sinais com falha gerados para o Fan End.")

print("\n--- Exemplo do DataFrame Final Gerado (Drive End) ---")
print(df_final_de.drop(columns=['sinal_final']).head())

Encontrados 8 sinais de operação normal para usar como base.

Processando base: 1730_Normal.npz (485643 pontos, 1730 RPM, Sensor Drive End)

Processando base: 1730_Normal.npz (485643 pontos, 1730 RPM, Sensor Fan End)

Processando base: 1750_Normal.npz (485063 pontos, 1750 RPM, Sensor Drive End)

Processando base: 1750_Normal.npz (485063 pontos, 1750 RPM, Sensor Fan End)

Processando base: 1772_Normal.npz (483903 pontos, 1772 RPM, Sensor Drive End)

Processando base: 1772_Normal.npz (483903 pontos, 1772 RPM, Sensor Fan End)

Processando base: 1797_Normal.npz (243938 pontos, 1797 RPM, Sensor Drive End)

Processando base: 1797_Normal.npz (243938 pontos, 1797 RPM, Sensor Fan End)


--- Geração e Combinação Concluídas! ---
Total de 84 sinais com falha gerados para o Drive End.
Total de 84 sinais com falha gerados para o Fan End.

--- Exemplo do DataFrame Final Gerado (Drive End) ---
  tipo_falha_adicionada   rpm  multiplicador_amplitude arquivo_normal_base  \
0         Pista Externa  1730  